<a href="https://colab.research.google.com/github/shreyus3003/Image-Classifi_seg_objDetec/blob/master/latest_imagenet_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy
import glob
import pylab as plt
import os
import keras.backend as K


# importing libraries
from matplotlib import pyplot
from keras import datasets
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# importing Image class from PIL package  
from PIL import Image  


from keras.callbacks import TensorBoard

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version
from PIL import Image


import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

#from kerastuner.tuners import RandomSearch
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from tensorboard.plugins.hparams import api as hp
from keras.utils import print_summary, to_categorical

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
from datetime import datetime
%load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir {logs_base_dir}

Using TensorFlow backend.


TensorFlow version:  2.2.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Unzipping the file 

!unzip /content/drive/My\ Drive/garbage-classification.zip

In [ ]:
# Importing and deriving the training images and labelling them
trash_images = []
labels = [] 
for fruit_dir_path in glob.glob("/content/garbage-classification/Garbage classification/Garbage classification/*/"):
    fruit_label = fruit_dir_path.split("/")[4]
    
    # Path for reading the files
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):

      # Using cv2 readig the images from path
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)

        # Resizing the images
        image = cv2.resize(image, (70, 70))

        # Colour Transformations 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        trash_images.append(image)
        labels.append(fruit_label)
        # Appending and deriving the final images 

trash_images = np.array(trash_images)
labels = np.array(labels)

In [ ]:
# Converting the Key-value dictonaries of the annotations into labels 

label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [ ]:
training_label_id = np.array([label_to_id_dict[i] for i in labels])

In [ ]:
# Checking the shape of the data 

print("Train_data shape......:",trash_images.shape)
print("Labels shape..........:",training_label_id.shape)

Train_data shape......: (2527, 70, 70, 3)
Labels shape..........: (2527,)


In [ ]:
# Splitting the data into Train and Test

(trainX, testX, trainY, testY) = train_test_split(trash_images, training_label_id, test_size=0.25)

In [ ]:
# Tensor Board Integration by giving the approriate log location with the time stamp 

now = datetime.now()
logdir = "logs/trash_images/" + now.strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
with file_writer.as_default():
  tf.summary.image("Training data", np.reshape(trainX[0:10],(-2,70,70,3)), step=0)
file_writer.set_as_default()

In [ ]:
# Path for the image folders

dir_path = '/content/garbage-classification/Garbage classification/Garbage classification'

In [ ]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [ ]:
# Checking the Length of the images

len(img_list)

2527

In [ ]:
## ---------Using Image Data Generator below code allows you to do the argumentation on the -----------------
##------------ mentioned parameters and sets your image positions---------------------

#_------------------- Generating for Train Image------------------
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

# ------------------Generating for test Image ------------------------

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(dir_path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory(dir_path,
                                        target_size=(300,300),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

#---------------------------Generating the validation dataset -------------------

validation_generator = test.flow_from_directory(
    dir_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)


Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [ ]:
# From the generator creating the labels and actual images 

for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 300, 300, 3), (32, 6))

In [ ]:
print (train_generator.class_indices)

Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(Labels)

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [ ]:
IMG_SHAPE = (224,224,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), #Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

#----------------------------TRAINING THE MODEL----------------------------------------------

history = model.fit_generator(train_generator, 
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs, 
                              workers=4,
                              validation_data=validation_generator, 
                              validation_steps=validation_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
71/71 [==============================] - 35s 493ms/step - loss: 0.9728 - accuracy: 0.6488 - val_loss: 0.7145 - val_accuracy: 0.7143
Epoch 2/10
71/71 [==============================] - 38s 540ms/step - loss: 0.5721 - accuracy: 0.7914 - val_loss: 0.5378 - val_accuracy: 0.7768
Epoch 3/10
71/71 [==============================] - 38s 540ms/step - loss: 0.4505 - accuracy: 0.8373 - val_loss: 0.7976 - val_accuracy: 0.7411
Epoch 4/10
71/71 [==============================] - 39s 547ms/step - loss: 0.3901 - accuracy: 0.8663 - val_loss: 0.5571 - val_accuracy: 0.7857
Epoch 5/10
71/71 [==============================] - 39s 546ms/step - loss: 0.3237 - accuracy: 0.8873 - val_loss: 0.4375 - val_accuracy: 0.8304
Epoch 6/10
71/71 [==============================] - 39s 549ms/step - loss: 0.2879 - accuracy: 0.8930 - val_loss: 0.4430 - val_accuracy: 0.8214
Epoch 7/10
71/71 [==============================] - 39s 549ms/step

In [ ]:


# SETTING THE SEEDS FOR RANDOM 
tf.random.set_seed(42)
np.random.seed(42)

model=tf.keras.Sequential()

#------------------------------CONVOLUTIONAL BLOCKS------------------------------

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2)) 

#--------------------------CLASSIFICATION LAYERS--------------------------------
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
#-----------------------------OUTPUT DENSE LAYER-------------------------------
model.add(tf.keras.layers.Dense(6,activation='softmax'))

# FILEPATH FOR SAVING THE MODEL LOGS
filepath="/content/trained_model.h5"
logdir="logs/trash_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#TENSORBOARD INTEGRATION
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# MODEL CHECK POINTS
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("trash_simple_model.h5", save_best_only=True)
callbacks_list = [model_checkpoint,tensorboard_callback]

In [ ]:
#---------------MODEL SUMMARY-------------
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43808)             0

In [ ]:
# ---------------PLOTTING THE MODEL AS PNG FILE ------------
keras.utils.plot_model(model, "simple_cnn_trash.png", show_shapes=True)

In [ ]:
# -------------------Compiling the model with categorical cross entropy as loss function and 
#. ADAM as optimiser with acuuracy and Mean Square Error and metrics---------------------------------------

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc','mse']) 

In [ ]:
#---------------- MODEL FITTING WITH EPOCHS, WORKERS, BY INITIATING THE CALL BACKS AND VALIDATION ON DATA---------------------

history = model.fit_generator(train_generator,
                              epochs=10,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

model.predict_generator(test_generator,callbacks=callbacks_list)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
71/71 [==============================] - 329s 5s/step - loss: 1.7062 - acc: 0.2727 - mse: 0.1339 - val_loss: 1.5460 - val_acc: 0.3170 - val_mse: 0.1243
Epoch 2/10
71/71 [==============================] - 330s 5s/step - loss: 1.6120 - acc: 0.3048 - mse: 0.1278 - val_loss: 1.5961 - val_acc: 0.3214 - val_mse: 0.1263
Epoch 3/10
68/71 [===========================>..] - ETA: 13s - loss: 1.5207 - acc: 0.3524 - mse: 0.1215

In [ ]:
#------------PRINTING THE LEARNING RATE USED----------------------
print(K.eval(model.optimizer.lr))


0.001
